In [6]:
import time
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt

from fbprophet import Prophet
from fbprophet.diagnostics import cross_validation
from fbprophet.diagnostics import performance_metrics
from fbprophet.plot import plot_cross_validation_metric
from sklearn.model_selection import ParameterGrid
import itertools

In [8]:
# prepare and clean data
df = pd.read_csv('data/train.csv', index_col=None)
df = df[['ID','Province_State','Date','Confirmed','Deaths']] #choose the columns needed
df['Date'] = pd.to_datetime(df['Date'])

# df1 for predicting confirmed cases
df1 = df.rename(columns = {'Date':'ds', 'Confirmed': 'y'})

# df2 for predicting death cases
df2 = df.rename(columns = {'Date':'ds', 'Deaths': 'y'})
df1,df2

(        ID Province_State         ds       y  Deaths
 0        0        Alabama 2020-04-12    3563      93
 1        1         Alaska 2020-04-12     272       8
 2        2        Arizona 2020-04-12    3542     115
 3        3       Arkansas 2020-04-12    1280      27
 4        4     California 2020-04-12   22795     640
 ...    ...            ...        ...     ...     ...
 7095  7095       Virginia 2020-08-31  120575    2580
 7096  7096     Washington 2020-08-31   74635    1915
 7097  7097  West Virginia 2020-08-31   10249     215
 7098  7098      Wisconsin 2020-08-31   75603    1122
 7099  7099        Wyoming 2020-08-31    3850      37
 
 [7100 rows x 5 columns],
         ID Province_State         ds  Confirmed     y
 0        0        Alabama 2020-04-12       3563    93
 1        1         Alaska 2020-04-12        272     8
 2        2        Arizona 2020-04-12       3542   115
 3        3       Arkansas 2020-04-12       1280    27
 4        4     California 2020-04-12      22795 

In [9]:
States = df['Province_State'].drop_duplicates()
len(States)

50

In [10]:
def generate_ForecastID(df_pred, target_str):
    '''
    df_pred: predicted data of one state from fbprophet 
    target_str: for renaming 'yhat'
    '''
    df = df_pred[['ds', 'yhat']]
    # select data after 2020-09-01
    df = df[df['ds'] >= '2020-09-01'].reset_index(drop = True)

    #push index to first column and use 'index' column to generate 'ForecastID'
    df = df.reset_index() 
    df['index'] = df['index']*50 + i

    df = df.rename(columns = {'index':'ForecastID', 'yhat': target_str }) 
    
    return df

In [11]:
def get_bestParameters(df, all_params):
    '''
    df: data of one state, has 'ds' and 'y'
    
    
    ## simple hyperparameter tuning
    '''
    mapes = []  # Store the RMSEs for each params here
    min_MAPE = 1000
    
    # Use cross validation to evaluate all parameters
    for params in all_params:
        m = Prophet(**params).fit(df)  # Fit model with given params
        df_cv = cross_validation(m, initial='30 days', period='30 days', horizon = '26 days')
        df_p = performance_metrics(df_cv, rolling_window=1)
        mape = df_p['mape'].values[0]
        mapes.append(mape)

        #find min mape and best parameters
        if min_MAPE > mape:
            min_MAPE = mape
            best_params = params

    # keep track of tuning results
    tuning_results = pd.DataFrame(all_params)
    tuning_results['mape'] = mapes
    
    return best_params
    

### Simple Model Fitting with default parameters

In [12]:
df1_all_states = [] #store predicted data for each state
df2_all_states = []
for i in range(len(States)):

    # training data: 2020-04-12 to 2020-08-31
    # predict confirmed cases
    df1_one_state = df1[df1['Province_State'] == States[i]]
    
    m1 = Prophet().fit(df1_one_state) # fit data to model
    future = m1.make_future_dataframe(periods=26, freq = 'D')
    forecast1 = m1.predict (future) # predict
    
    df1_one_state_pred = generate_ForecastID(forecast1, 'Confirmed')  #generate ForecastID
    df1_all_states.append(df1_one_state_pred)
    
    # predict death cases
    df2_one_state = df2[df2['Province_State'] == States[i]]
    
    m2 = Prophet().fit(df2_one_state)
    forecast2 = m2.predict (future)
    
    df2_one_state_pred = generate_ForecastID(forecast2, 'Deaths') 
    df2_all_states.append(df2_one_state_pred)

INFO:numexpr.utils:NumExpr defaulting to 8 threads.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disab

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seaso

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seaso

In [17]:
df1_pred = pd.concat(df1_all_states, ignore_index=True).sort_values(by=['ForecastID'])
df1_pred = df1_pred.drop(columns = ['ds'])
df1_pred

,ForecastID,Confirmed
0,0,126675.162456
26,1,5361.346132
52,2,205086.553878
78,3,61866.576130
104,4,731774.511300
...,...,...
1195,1295,146164.716040
1221,1296,90550.499395
1247,1297,13205.743116
1273,1298,95292.083386


In [18]:
df2_pred = pd.concat(df2_all_states, ignore_index=True).sort_values(by=['ForecastID'])
df2_pred

,ForecastID,ds,Deaths
0,0,2020-09-01,2189.250984
26,1,2020-09-01,36.775603
52,2,2020-09-01,5146.870928
78,3,2020-09-01,787.889615
104,4,2020-09-01,13270.892751
...,...,...,...
1195,1295,2020-09-26,2902.106764
1221,1296,2020-09-26,2257.703536
1247,1297,2020-09-26,293.430870
1273,1298,2020-09-26,1298.615939


In [21]:
df1_pred['Deaths'] = df2_pred['Deaths']
df1_pred

,ForecastID,Confirmed,Deaths
0,0,126675.162456,2189.250984
26,1,5361.346132,36.775603
52,2,205086.553878,5146.870928
78,3,61866.576130,787.889615
104,4,731774.511300,13270.892751
...,...,...,...
1195,1295,146164.716040,2902.106764
1221,1296,90550.499395,2257.703536
1247,1297,13205.743116,293.430870
1273,1298,95292.083386,1298.615939


In [22]:
df1_pred.to_csv('output/submission1.csv', index = False)

### Hyperparameter Tuning

In [23]:
# Simple Version
# Generate all combinations of parameters
param_grid = {  
    'changepoint_prior_scale': [0.001, 0.01, 0.05, 0.1, 0.5], #default 0.05
    'seasonality_prior_scale': [0.01, 0.1, 1.0, 10.0], #default 10
}
all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]

In [24]:
# Generate more model
# Generate all combinations of parameters
from sklearn.model_selection import ParameterGrid
params_grid = {'seasonality_mode':['multiplicative','additive'], 
               'changepoint_prior_scale': [0.001, 0.002, 0.005, 0.05, 0.5],  # default 0.05, reasonable range [0.001, 0.5]
               'seasonality_prior_scale': [0.01, 0.02, 0.05, 0.1, 1, 10.0],  # default 10, reasonable range [0.01, 10]
               'changepoint_range' : [0.8, 0.85, 0.9, 0.95] 
              }

all_params = ParameterGrid(params_grid)
cnt = 0
for params in all_params:
    #print(params)
    cnt = cnt+1
    
print('Total Possible Models',cnt)

Total Possible Models 240


**NOTE**

Since we have possible models upto 240, we need to run 240 * 3(cross validation) * 2 (cases/death cases) * 50 (states). Approximately 33 hours in total. So we might want to break up the following for loop into several one and run on differnt kernels to save time.

In [ ]:
df1_all_states = [] #store predicted data for each state
df2_all_states = []
for i in range(len(States)):

    # training data: 2020-04-12 to 2020-08-31
    # predict confirmed cases
    df1_one_state = df1[df1['Province_State'] == States[i]]
    
    best_params1 = get_bestParameters(df1_one_state, all_params)
    m1 = Prophet(**best_params1).fit(df1_one_state) # fit data to model
    future = m1.make_future_dataframe(periods=26, freq = 'D')
    forecast1 = m1.predict (future) # predict
    
    df1_one_state_pred = generate_ForecastID(forecast1, 'Confirmed')  #generate ForecastID
    df1_all_states.append(df1_one_state_pred)
    
    # predict death cases
    df2_one_state = df2[df2['Province_State'] == States[i]]
    
    best_params2 = get_bestParameters(df2_one_state, all_params)
    m2 = Prophet(**best_params2).fit(df2_one_state)
    forecast2 = m2.predict (future)
    
    df2_one_state_pred = generate_ForecastID(forecast2, 'Deaths') 
    df2_all_states.append(df2_one_state_pred)

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Making 3 forecasts with cutoffs between 2020-06-06 00:00:00 and 2020-08-05 00:00:00


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


INFO:fbprophet:Making 3 forecasts with cutoffs between 2020-06-06 00:00:00 and 2020-08-05 00:00:00


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


INFO:fbprophet:Making 3 forecasts with cutoffs between 2020-06-06 00:00:00 and 2020-08-05 00:00:00


...

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Making 3 forecasts with cutoffs between 2020-06-06 00:00:00 and 2020-08-05 00:00:00


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Making 3 forecasts with cutoffs between 2020-06-06 00:00:00 and 2020-08-05 00:00:00


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


INFO:fbprophet:Making 3 forecasts with cutoffs between 2020-06-06 00:00:00 and 2020-08-05 00:00:00


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Making 3 forecasts with cutoffs between 2020-06-06 00:00:00 and 2020-08-05 00:00:00


In [15]:
df1_pred = pd.concat(df1_all_states, ignore_index=True).sort_values(by=['ForecastID'])
df2_pred = pd.concat(df2_all_states, ignore_index=True).sort_values(by=['ForecastID'])
df1_pred['Deaths'] = df2_pred['Deaths']
df1_pred = df1_pred.drop(columns = ['ds'])
df1_pred

ValueError: No objects to concatenate

In [ ]:
df1_pred.to_csv('output/submission3.csv', index = False)